In [1]:
import jax
import jax.numpy as jnp
from dataclasses import dataclass
import inspeqtor as sq
from qiskit_ibm_runtime.fake_provider import FakeJakartaV2
from pathlib import Path
import receipt_utils

In [2]:
@dataclass
class CompackConfig:
    identifier: str
    shots: int
    sample_size: int
    qubit_idx: int

In [3]:
SHOTS = 3000
SAMPLE_SIZE = 1000
qubit_idx = 2
test_cases = [
    CompackConfig(identifier="q2", shots=SHOTS, sample_size=SAMPLE_SIZE, qubit_idx=qubit_idx),
]

# INSTANCE = "ibm_open/open_pulse/specq"
# backend = FakeJakartaV2()
# service = None
# BACKEND_NAME = backend.name

INSTANCE = "utokyo-kawasaki/keio-internal/keio-students"
# INSTANCE = "ibm-q-utokyo/keio-internal/keio-students"
BACKEND_NAME = "ibm_kawasaki"
# BACKEND_NAME = "ibm_kyiv"
service, backend = sq.qiskit.get_ibm_service_and_backend(instance=INSTANCE, backend_name=BACKEND_NAME)

qubit_indices = [test_case.qubit_idx for test_case in test_cases]

backend_properties = sq.qiskit.IBMQDeviceProperties.from_backend(
    backend=backend, qubit_indices=qubit_indices, service=service
)

# pulse_sequence = sq.utils.predefined.get_multi_drag_pulse_sequence_v2(drag_version=2)
# pulse_sequence = sq.utils.predefined.get_multi_drag_pulse_sequence_v2(drag_version=2, min_beta=0.0, max_beta=0.0)
pulse_sequence = sq.utils.predefined.get_multi_drag_pulse_sequence_v3()

# Initialize configutations
configs: list[sq.data.ExperimentConfiguration] = []
for compact_config in test_cases:
    config = sq.data.ExperimentConfiguration(
        qubits=[backend_properties.qubit_informations[0]],
        expectation_values_order=sq.constant.default_expectation_values_order,
        parameter_names=pulse_sequence.get_parameter_names(),
        backend_name=backend_properties.name,
        shots=compact_config.shots,
        EXPERIMENT_IDENTIFIER=compact_config.identifier,
        EXPERIMENT_TAGS=[compact_config.identifier],
        description="The experiment to test random drag sequence",
        device_cycle_time_ns=backend_properties.dt,
        sequence_duration_dt=pulse_sequence.pulse_length_dt,
        instance=INSTANCE,
        sample_size=compact_config.sample_size,
        additional_info={
            "MASTER_KEY_SEED": compact_config.qubit_idx,
            "TARGET_GATE": "SX_v2",
        },
    )
    configs.append(config)

# path = Path(f"./0001_{BACKEND_NAME}_test")
path = Path(f"./0005_{BACKEND_NAME}_paper_1_sx_v2")
# Save configurations and pulse sequence
for config in configs:
    config.to_file(path / f"{config.EXPERIMENT_IDENTIFIER}")
    pulse_sequence.to_file(path / f"{config.EXPERIMENT_IDENTIFIER}")

In [4]:
receipt_utils.submit_experiment_command(
    path
)

'python receipt.py prepare-parallel-experiment 0005_ibm_kawasaki_paper_1_sx_v2/ && python receipt.py execute-parallel-experiment 0005_ibm_kawasaki_paper_1_sx_v2/'

In [10]:
receipt_utils.analyze_experiment_command(
    path
)

'python receipt.py wait-for-jobs 0003_ibm_kawasaki_paper_1_sx_v2/ --wait-interval=600 && python receipt.py train-model 0003_ibm_kawasaki_paper_1_sx_v2/ && python receipt.py batch-benchmark-gate 0003_ibm_kawasaki_paper_1_sx_v2/q0/ 0003_ibm_kawasaki_paper_1_sx_v2/q0/ckpts/'

In [5]:
receipt_utils.test_receipt_command(
    path
)

'python receipt.py prepare-parallel-experiment 0001_fake_jakarta_test/ && python receipt.py execute-parallel-experiment 0001_fake_jakarta_test/ --wait && python receipt.py train-model 0001_fake_jakarta_test/'

In [9]:
submit_command = f"python receipt.py prepare-parallel-experiment {path} && \
    python receipt.py execute-parallel-experiment {path}"

retrive_command = f"python receipt.py wait-for-jobs {path}/ --wait-interval=600 && \
    python receipt.py train-model {path} MAEF && \
    python receipt.py train-model {path} MSEE && \
    python receipt.py batch-benchmark-gate {path}/q{qubit_idx}/ {path}/q{qubit_idx}/ckpts/MAEF && \
    python receipt.py batch-benchmark-gate {path}/q{qubit_idx}/ {path}/q{qubit_idx}/ckpts/MSEE"


python receipt.py prepare-parallel-experiment 0004_ibm_kawasaki_paper_1_sx_v2 &&     python receipt.py execute-parallel-experiment 0004_ibm_kawasaki_paper_1_sx_v2


In [10]:
submit_command

'python receipt.py prepare-parallel-experiment 0004_ibm_kawasaki_paper_1_sx_v2 &&     python receipt.py execute-parallel-experiment 0004_ibm_kawasaki_paper_1_sx_v2'

In [11]:
retrive_command

'python receipt.py wait-for-jobs 0004_ibm_kawasaki_paper_1_sx_v2/ --wait-interval=600 &&     python receipt.py train-model 0004_ibm_kawasaki_paper_1_sx_v2 MAEF &&     python receipt.py train-model 0004_ibm_kawasaki_paper_1_sx_v2 MSEE &&     python receipt.py batch-benchmark-gate 0004_ibm_kawasaki_paper_1_sx_v2/q2/ 0004_ibm_kawasaki_paper_1_sx_v2/q2/ckpts/MAEF &&     python receipt.py batch-benchmark-gate 0004_ibm_kawasaki_paper_1_sx_v2/q2/ 0004_ibm_kawasaki_paper_1_sx_v2/q2/ckpts/MSEE'